In [1]:
from bigbird.core import modeling
import tensorflow.compat.v2 as tf
from tqdm import tqdm
from transformers import BigBirdPegasusConfig, BigBirdPegasusModel
import torch
import numpy as np

tf.enable_v2_behavior()

# TODO
# "type_vocab_size": 1

In [2]:
def difference_between_tensors(tf_tensor, pt_tensor):
    tf_np = np.array(tf_tensor)
    pt_np = np.array(pt_tensor.detach())
    return np.max(np.abs(tf_np - pt_np))

TF_CKPT_DIR = "tf_ckpt/bigbird-pegasus-large-arxiv/model.ckpt-0"
HF_CKPT_DIR = "google/bigbird-pegasus-large-arxiv/pytorch_model.bin"



In [19]:
# 'couple_encoder_decoder' is switching pegasus & encoder-decoder

bbc = {
      # transformer basic configs
      "couple_encoder_decoder": False,
      "vocab_size": 96103,
      "attention_probs_dropout_prob": 0.0,
      "hidden_act": "gelu",
      "hidden_dropout_prob": 0.0,
      "hidden_size": 1024,
      "initializer_range": 0.02,
      "intermediate_size": 4096,
      "max_position_embeddings": 4096,
      "num_attention_heads": 16,
      "num_hidden_layers": 16,
      "num_decoder_layer": 16,
      "type_vocab_size": 1,
      "use_bias": True,
      "rescale_embedding": False,
      "scope": "pegasus",
      # sparse mask configs
      "attention_type": "block_sparse", # "block_sparse" "original_full"
      "norm_type": "prenorm",
      "block_size": 16,
      "num_rand_blocks": 3,
      # common bert configs
      "max_encoder_length": 128,
      "max_decoder_length": 16,
      "batch_size": 1,
      "beam_size": 5, #
      "alpha": 0.1, #
}
hf_bigbird_config = BigBirdPegasusConfig(
        vocab_size=bbc['vocab_size'],
        max_position_embeddings=bbc["max_position_embeddings"],
        encoder_layers=bbc["num_hidden_layers"],
        encoder_ffn_dim=bbc["intermediate_size"],
        encoder_attention_heads=bbc['num_attention_heads'],
        decoder_layers=bbc['num_decoder_layer'],
        decoder_ffn_dim=bbc["intermediate_size"],
        decoder_attention_heads=bbc['num_attention_heads'],
        encoder_layerdrop=0.0,
        decoder_layerdrop=0.0,
        use_cache=True,
        is_encoder_decoder=True,
        activation_function="gelu_fast",
        d_model=bbc['hidden_size'],
        dropout=0.1,
        attention_dropout=0.0,
        activation_dropout=0.0,
        init_std=bbc["initializer_range"],
        decoder_start_token_id=2,
        classifier_dropout=0.0,
        scale_embedding=bbc['rescale_embedding'],
        gradient_checkpointing=False,
        pad_token_id=1,
        bos_token_id=0,
        eos_token_id=2,
        attention_type=bbc['attention_type'], # only for encoder
        block_size=bbc['block_size'],
        num_random_blocks=bbc['num_rand_blocks'],
        use_bias=bbc['use_bias'],
)

bigbird_config = bbc

In [20]:
s1 = bigbird_config["batch_size"]
s2 = bigbird_config["max_encoder_length"]
s3 = bigbird_config["max_decoder_length"]

np.random.seed(0)
arr = np.random.randint(1, s2, size=s1*s2).reshape(s1, s2)
input_ids = tf.convert_to_tensor(arr, dtype=tf.int32)
hf_input_ids = torch.from_numpy(arr).long()

np.random.seed(0)
arr = np.random.randint(1, s3, size=s1*s3).reshape(s1, s3)
target_ids = tf.convert_to_tensor(arr, dtype=tf.int32)
hf_target_ids = torch.from_numpy(arr).long()

In [21]:
target_ids

<tf.Tensor: shape=(1, 16), dtype=int32, numpy=
array([[13,  6,  1,  4, 12,  4,  8, 10,  4,  6,  3,  5,  8,  7,  9,  9]],
      dtype=int32)>

In [22]:
input_ids

<tf.Tensor: shape=(1, 128), dtype=int32, numpy=
array([[ 45,  48, 118,  65,  68, 124,  68, 104,  10,  84,  22, 115,  37,
         88,  71,  89,  89,  13,  59,  66, 103,  40,  88,  47,  89,  82,
         38,  26,  78,  73,  10,  21, 116,  81, 116,  70, 127,  80,  48,
         65,  83, 100,  89,  50, 116,  30,  20,  20,  15,  40,  33,  66,
         10,  58,  33,  32,  75, 117,  24,  36, 127,  76, 115,  56,  29,
         35,   1,   1,  37,  54,   6,  39, 105, 117,  18,  80,   5, 106,
         43,  59,  32, 121,   2,  66, 104,  42,  58,  36, 103, 120,  12,
         47,  83,  92,   1,  15, 100,  54,  13, 122,  43,  85,  76,  69,
          7,  69,  48, 117,   4,  77, 101,  53, 105,  79,  16,  21, 100,
         59,  24,  80,  14, 118,  86,  49,  50,  70,  42,  36]],
      dtype=int32)>

In [23]:
model = modeling.TransformerModel(bigbird_config)
o = model(input_ids, target_ids=target_ids)
del o

ckpt_reader = tf.compat.v1.train.NewCheckpointReader(TF_CKPT_DIR)
model.set_weights([ckpt_reader.get_tensor(v.name[:-2]) for v in tqdm(model.trainable_weights, position=0)])

INFO:absl:**** Using block sparse attention ****
INFO:absl:**** Using block sparse attention ****
INFO:absl:**** Using block sparse attention ****
INFO:absl:**** Using block sparse attention ****
INFO:absl:**** Using block sparse attention ****
INFO:absl:**** Using block sparse attention ****
INFO:absl:**** Using block sparse attention ****
INFO:absl:**** Using block sparse attention ****
INFO:absl:**** Using block sparse attention ****
INFO:absl:**** Using block sparse attention ****
INFO:absl:**** Using block sparse attention ****
INFO:absl:**** Using block sparse attention ****
INFO:absl:**** Using block sparse attention ****
INFO:absl:**** Using block sparse attention ****
INFO:absl:**** Using block sparse attention ****
INFO:absl:**** Using block sparse attention ****
INFO:absl:**** Using original full attention ****
INFO:absl:**** Using original full attention ****
INFO:absl:**** Using original full attention ****
INFO:absl:**** Using original full attention ****
INFO:absl:**** U

In [8]:
a = set([v.name[:-2] for v in model.trainable_variables])
b = set([b[0] for b in tf.train.list_variables(TF_CKPT_DIR)])

In [4]:
tf.train.list_variables(TF_CKPT_DIR)

[('global_step', []),
 ('pegasus/decoder/LayerNorm/beta', [1024]),
 ('pegasus/decoder/LayerNorm/gamma', [1024]),
 ('pegasus/decoder/layer_0/attention/encdec/LayerNorm/beta', [1024]),
 ('pegasus/decoder/layer_0/attention/encdec/LayerNorm/gamma', [1024]),
 ('pegasus/decoder/layer_0/attention/encdec/key/bias', [1024]),
 ('pegasus/decoder/layer_0/attention/encdec/key/kernel', [1024, 1024]),
 ('pegasus/decoder/layer_0/attention/encdec/query/bias', [1024]),
 ('pegasus/decoder/layer_0/attention/encdec/query/kernel', [1024, 1024]),
 ('pegasus/decoder/layer_0/attention/encdec/value/bias', [1024]),
 ('pegasus/decoder/layer_0/attention/encdec/value/kernel', [1024, 1024]),
 ('pegasus/decoder/layer_0/attention/encdec_output/dense/bias', [1024]),
 ('pegasus/decoder/layer_0/attention/encdec_output/dense/kernel',
  [1024, 1024]),
 ('pegasus/decoder/layer_0/attention/output/dense/bias', [1024]),
 ('pegasus/decoder/layer_0/attention/output/dense/kernel', [1024, 1024]),
 ('pegasus/decoder/layer_0/attenti